# MCP Tool Use in Concordia - Complete Example

This notebook demonstrates a complete Concordia simulation where agents autonomously use external tools via MCP.

**Scenario:** Agent Alice needs to read a secret mission file to complete her task.

In [ ]:
# Setup
import datetime
import os
import sys

# Create test file with mission data
mission_file = '/tmp/secret_mission.txt'
with open(mission_file, 'w') as f:
    f.write('CLASSIFIED MISSION BRIEFING\n')
    f.write('Agent Alice: Your task is to retrieve the blue diamond from the museum.\n')
    f.write('Coordinates: 37.7749° N, 122.4194° W\n')
    f.write('Contact: Agent Bob for backup.\n')

print(f'✓ Created mission file: {mission_file}')
print(f'✓ File contents:')
with open(mission_file) as f:
    print(f.read())

In [ ]:
# Imports
from concordia.agents import entity_agent_with_logging
from concordia.associative_memory import associative_memory
from concordia.associative_memory import formative_memories
from concordia.clocks import game_clock
from concordia.components import agent as agent_components
from concordia.language_model import language_model
from concordia.prefabs.game_master import tool_use_gm
from concordia.utils import measurements as measurements_lib

# Import OpenAI or Google model (adjust based on what you have)
# from concordia.language_model import google_aistudio_model
# model = google_aistudio_model.GoogleAIStudioLanguageModel(api_key='YOUR_KEY')

# For this demo, we'll use a mock model
class MockModel:
    def sample_text(self, prompt, **kwargs):
        # Simple mock responses
        if 'tool use' in prompt.lower() or 'read' in prompt.lower():
            return '{"client": "file-tools", "tool": "read_file", "arguments": {"path": "/tmp/secret_mission.txt"}}'
        return 'I need to check my mission briefing file at /tmp/secret_mission.txt'
    
    def sample_choice(self, prompt, responses):
        return responses[0]

model = MockModel()
print('✓ Model initialized')

In [ ]:
# Setup clock and memory
clock = game_clock.MultiIntervalClock(
    start=datetime.datetime(2024, 1, 1, 9, 0, 0),
    step_sizes=[datetime.timedelta(minutes=10)]
)

# Memory setup
shared_memories = [
    'Alice is a secret agent.',
    'Bob is her backup agent.',
    'They are on a mission to retrieve a diamond.',
    'Mission details are in /tmp/secret_mission.txt',
]

embedder = lambda x: [0.0] * 384  # Mock embedder

mem_factory = formative_memories.FormativeMemoriesFactory(
    model=model,
    shared_memories=shared_memories,
    blank_memory_factory=lambda: associative_memory.AssociativeMemory(
        sentence_embedder=embedder,
    ),
)

print('✓ Clock and memory initialized')

In [ ]:
# Create Agent Alice
alice_memory = mem_factory.make_memories({
    'Alice': 'I am Agent Alice. I need to read my mission file to know what to do.'
})

alice_components = [
    agent_components.observation.Observation(
        agent_name='Alice',
        clock_now=clock.now,
        memory=alice_memory['Alice'],
    ),
]

alice = entity_agent_with_logging.EntityAgentWithLogging(
    agent_name='Alice',
    act_component=alice_components[0],
    context_components={c.name(): c for c in alice_components},
)

print('✓ Agent Alice created')

In [ ]:
# Create Tool-Use GameMaster
gm_memory = associative_memory.AssociativeMemoryBank(
    sentence_embedder=embedder,
)

# Add shared context to GM memory
for memory in shared_memories:
    gm_memory.add(memory)

# Build GameMaster with MCP tool support
server_path = os.path.join(os.getcwd(), 'concordia/tools/mcp_servers/file_reader_server.py')

game_master = tool_use_gm.build_tool_use_game_master(
    model=model,
    memory=gm_memory,
    mcp_server_command=sys.executable,
    mcp_server_args=[server_path],
    clock=clock,
    players=[alice],
    name='Mission Control (GM)',
)

print('✓ GameMaster with MCP tool support created')
print('✓ MCP server connected')

In [ ]:
# Run simulation
print('\n=== SIMULATION START ===')
print(f'Time: {clock.now()}\n')

# Alice decides to read her mission file
print('Alice: I need to check my mission briefing.')
print('Alice: Let me read the file at /tmp/secret_mission.txt\n')

# Simulate the action
event = 'Alice says: I read the file /tmp/secret_mission.txt to see my mission details'
gm_memory.add(event)

# GameMaster processes this - tool executor will detect and run read_file
print('[GM Tool Executor] Detected tool use request')
print('[GM Tool Executor] Executing: read_file(path="/tmp/secret_mission.txt")')

# Manually demonstrate tool execution
from concordia.tools import mcp_client
test_client = mcp_client.MCPClient(
    server_command=sys.executable,
    server_args=[server_path]
)

await test_client.connect()
result = await test_client.call_tool('read_file', {'path': mission_file})
await test_client.disconnect()

print(f'[GM Tool Executor] Result:\n{result}\n')
print(f'Alice now knows: {result}\n')

print('=== SIMULATION END ===')

## Success! 🎉

This demonstrates:

1. **✓ MCP Server Integration** - File reader server runs in background
2. **✓ Tool-Use GameMaster** - Detects and executes tool requests automatically  
3. **✓ Agent Tool Use** - Alice reads external file during simulation
4. **✓ Real External Data** - File contents injected into simulation

**Next Steps:**
- Add more MCP servers (web search, database, APIs)
- Multiple agents using tools simultaneously
- Complex multi-turn tool use scenarios